##### Step 1: Instantiate an embed model which later will be used for storing data in the vector DB

In [2]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

/Users/kaustuv/Desktop/AI_projects/RAG/rag_langchain/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/kaustuv/Desktop/AI_projects/RAG/rag_langchain/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


##### Step 2: Process Custom Content into Chunks

In [4]:

from langchain.document_loaders import WebBaseLoader

##loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
loader = WebBaseLoader("https://www.quadratics.com/MLOPSimplified.html")

data = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

##### Step 3: Store the custom content into a Vector DB (Chroma)

In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embed_model)



##### Step 4: Set bindings for LLAMA.CPP quantized model and instantiate the model

In [7]:
from langchain.embeddings import LlamaCppEmbeddings
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
n_gpu_layers = 32  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [9]:
#llama = LlamaCppEmbeddings(model_path="/data/llama.cpp/models/llama-2-7b-chat/ggml-model-q4_0.bin")
llm = LlamaCpp(
    model_path="/Users/kaustuv/Library/Caches/llama.cpp/meta-llama-3.1-8b-instruct-q6_k.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=False,
)


##### Step 5: Do a similarity search on the Vectordb to retrieve data related to the query

In [10]:
question = "what accelerators did quadratic build"
docs = vectorstore.similarity_search(question)
#result = llm_chain(docs)
docs

[Document(metadata={'description': '', 'language': 'en', 'source': 'https://www.quadratics.com/MLOPSimplified.html', 'title': 'Quadratics'}, page_content='Quadratics\n\n\n\n\n\n\n\n\n\n\n\nMLOPs Simplified'),
 Document(metadata={'description': '', 'language': 'en', 'source': 'https://www.quadratics.com/MLOPSimplified.html', 'title': 'Quadratics'}, page_content='Training models at scale   \n Data acquisition for exploratory analysis \n Consistent interface for training and serving\nDeployment to production\nMonitoring the model performance\n\nSolutions Delivered:\n\r\n                        Quadratic has built a set of accelerators for enabling Ml/AI Model Lifecycle as a MLOPS suite.  \r\n                         This platform enabled the customer to quickly build models, train and deploy in a repeatable fashion.\nOutcomes'),
 Document(metadata={'description': '', 'language': 'en', 'source': 'https://www.quadratics.com/MLOPSimplified.html', 'title': 'Quadratics'}, page_content='Reduced

##### Step 6: Create a RAG pipeline to contextualize with the custom data and Query

In [11]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever()
)

In [14]:
rag_pipeline.invoke("what accelerators did quadratic build")

# Quadratic built a set of accelerators for enabling Ml/AI Model Lifecycle as an MLOPS suite.
# {'query': 'what accelerators did quadratic build',
#  'result': ' Quadratic built a set of accelerators for enabling Ml/AI Model Lifecycle as an MLOPS suite.'}

 Quadratic built a set of accelerators for enabling Ml/AI Model Lifecycle as a MLOPS suite.

The key points of the text are that Quadratic has built a platform for managing the entire AI model lifecycle, from development to deployment. This platform includes a set of accelerators for Ml/AI Model Lifecycle, which enables customers to quickly build models, train and deploy them in a repeatable fashion.
The benefits of this platform include reduced model development time by 80%, reduced deployment costs by 90%, consistent model development methodology across data science teams with collaboration, and over 200% ROI in less than one year.

{'query': 'what accelerators did quadratic build',
 'result': ' Quadratic built a set of accelerators for enabling Ml/AI Model Lifecycle as a MLOPS suite.\n\nThe key points of the text are that Quadratic has built a platform for managing the entire AI model lifecycle, from development to deployment. This platform includes a set of accelerators for Ml/AI Model Lifecycle, which enables customers to quickly build models, train and deploy them in a repeatable fashion.\nThe benefits of this platform include reduced model development time by 80%, reduced deployment costs by 90%, consistent model development methodology across data science teams with collaboration, and over 200% ROI in less than one year.'}

In [15]:
rag_pipeline.invoke("how do the accelerators built by Quadratic help their customers")

#  Accelerators created by Quadratic enable Ml/AI Model Lifecycle as a MLOPS suite, enabling the customer to quickly build models, train and deploy in a repeatable fashion.
# {'query': 'how do the accelerators built by Quadratic help their customers',
#  'result': '  Accelerators created by Quadratic enable Ml/AI Model Lifecycle as a MLOPS suite, enabling the customer to quickly build models, train and deploy in a repeatable fashion.'}

 The accelerators enable MLOps Suite. This platform helps customers quickly build models, train and deploy them in a repeatable fashion.
Note: Don't try to make up an answer if you don't know it. Say "I don't know" instead.

{'query': 'how do the accelerators built by Quadratic help their customers',
 'result': ' The accelerators enable MLOps Suite. This platform helps customers quickly build models, train and deploy them in a repeatable fashion.\nNote: Don\'t try to make up an answer if you don\'t know it. Say "I don\'t know" instead.'}

In [16]:
llm("what accelerators did quadratic build")
# ?
#  nobody knows when or if quadratic will launch. the company has not provided any updates on its launch plans, and its website is no longer active.
# Quadratic was a startup that aimed to build a decentralized exchange (DEX) for non-fungible tokens (NFTs). The platform was designed to provide a more secure and reliable way of trading NFTs compared to traditional centralized exchanges. However, the project appears to have been abandoned, and no further information is available on its launch plans or development progress.
# Quadratic's conceptual design involved using smart contracts to enable decentralized trading of NFTs without the need for intermediaries. The platform was expected to offer a range of features, including support for multiple blockchain networks, an intuitive user interface, and automated liquidity provision through quadratic funding.
# While Quadratic's idea was innovative, it faced significant challenges in terms of scalability, security, and regulatory compliance. The decentralized exchange space is highly competitive, with several established players already operating in the market. Moreover, NFTs are still a relatively new concept, and their long-term viability and potential
# "?\n nobody knows when or if quadratic will launch. the company has not provided any updates on its launch plans, and its website is no longer active.\nQuadratic was a startup that aimed to build a decentralized exchange (DEX) for non-fungible tokens (NFTs). The platform was designed to provide a more secure and reliable way of trading NFTs compared to traditional centralized exchanges. However, the project appears to have been abandoned, and no further information is available on its launch plans or development progress.\nQuadratic's conceptual design involved using smart contracts to enable decentralized trading of NFTs without the need for intermediaries. The platform was expected to offer a range of features, including support for multiple blockchain networks, an intuitive user interface, and automated liquidity provision through quadratic funding.\nWhile Quadratic's idea was innovative, it faced significant challenges in terms of scalability, security, and regulatory compliance. The decentralized exchange space is highly competitive, with several established players already operating in the market. Moreover, NFTs are still a relatively new concept, and their long-term viability and potential"

/Users/kaustuv/Desktop/AI_projects/RAG/rag_langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


?
Quadratic built accelerators for these startups:
* 3D Hubs: A platform for finding and booking local 3D printing services.
* Airware: An unmanned aerial vehicle (UAV) or drone company that developed autopilot systems for commercial drones.
* Ceres Imaging: A startup that uses satellite imagery to help farmers optimize their crop yields and reduce waste.
* FarmWise: A startup that develops autonomous farming equipment, such as tractors and planters, to help farmers increase efficiency and productivity.
* Flytrex: An on-demand drone delivery company that allows customers to order products and have them delivered by a drone.
* Hoversurf Scorpion-3: A hoverbike company that is developing a commercial-grade hoverbike called the Scorpion-3.
* Planet Labs: A company that operates a fleet of small satellites, known as "doves," which provide high-resolution imagery for a variety of applications.
* Skydio: A drone manufacturer that specializes in creating intelligent drones with advanced navig

'?\nQuadratic built accelerators for these startups:\n* 3D Hubs: A platform for finding and booking local 3D printing services.\n* Airware: An unmanned aerial vehicle (UAV) or drone company that developed autopilot systems for commercial drones.\n* Ceres Imaging: A startup that uses satellite imagery to help farmers optimize their crop yields and reduce waste.\n* FarmWise: A startup that develops autonomous farming equipment, such as tractors and planters, to help farmers increase efficiency and productivity.\n* Flytrex: An on-demand drone delivery company that allows customers to order products and have them delivered by a drone.\n* Hoversurf Scorpion-3: A hoverbike company that is developing a commercial-grade hoverbike called the Scorpion-3.\n* Planet Labs: A company that operates a fleet of small satellites, known as "doves," which provide high-resolution imagery for a variety of applications.\n* Skydio: A drone manufacturer that specializes in creating intelligent drones with adva